### Langchain Agent example with OCI Meta model
Using the old version of tools


In [1]:
%%capture
!pip install langchain_community
!pip install tavily-python
!pip install duckduckgo-search

In [10]:
from langchain_community.chat_models import ChatOCIGenAI
from langchain_core.messages import HumanMessage
from langchain.agents import initialize_agent, Tool
import time
from tavily import TavilyClient
import os

# Tavily config API
tavily_client = TavilyClient(api_key='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') # Essa chave tem um limite de uso, mas funciona e é para você usar gratuitamente tbm!

# Função para realizar a busca
def search_web(query):
    try:
        response = tavily_client.search(query=query,include_domains=["webmd.com"])
        return response
    except Exception as e:
        print(f"Ocorreu um erro ao realizar a busca: {e}")
        return None

# Configura a tool, ela pode ser qualquer função python
# não funciona bem com funções com mais de um argumento como def function(a, b)
search_tool = Tool(
    name="Pesquisa na Web",
    func=search_web,
    description="Realiza uma pesquisa na web para obter informações sobre saúde e bem estar."
)

# Informações de autenticação (substituia pelas suas!)
COMPARTMENT_ID = "ocid1.tenancy.oc1..aaaaaaaakzuzfystmhyj7je5x7ymwd3ofd7wb6rtsfrx7nvbo272vta3rwna" 
AUTH_TYPE = "API_KEY"
CONFIG_PROFILE = "CHICAGO"
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

# Inicializa o modelo na GenAI
chat = ChatOCIGenAI(
  model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyarojgfh6msa452vziycwfymle5gxdvpwwxzara53topmq",
  service_endpoint=endpoint,
  compartment_id=COMPARTMENT_ID,
  provider="meta",
  model_kwargs={
    "temperature": 0.5,
    "max_tokens": 600,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "top_p": 0.75
  },
  auth_type=AUTH_TYPE,
  auth_profile=CONFIG_PROFILE
)
tools=[search_tool]
tool_names = ['search_tool']

# Inicializa o agente
agent = initialize_agent(
    tools=[search_tool],
    llm=chat,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

############# Input
user_input = "Estou com dor no joelho"

############ Prompt
prompt = f"""Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin! Remember to answer as a compansionate medical professional when giving your final answer.
    Do always think and answer in portuguese
    Query: {user_input}"""


# Executar o agente 
response = agent.invoke(prompt)


print("************************** Resposta **************************")
print(response['output'])




> Entering new AgentExecutor chain...
Question: Estou com dor no joelho
Thought: Preciso entender melhor o que pode estar causando essa dor no joelho. É importante considerar várias possibilidades, desde lesões até condições crônicas. Vou começar realizando uma pesquisa para obter informações sobre as causas comuns de dor no joelho.

Action: Pesquisa na Web
Action Input: dor no joelho causas comuns
Observation: {'query': 'dor no joelho causas comuns', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'WebMD Joint pain, Joint pain and Weakness: Common Related Medical Conditions', 'url': 'https://symptomchecker.webmd.com/multiple-symptoms?locations=44|47|47&symptomids=279|279|6&symptoms=joint-pain|joint-pain|weakness', 'content': 'WebMD Symptom Checker helps you find the most common medical conditions indicated by the symptoms Joint pain, Joint pain and Weakness and including Carpal tunnel syndrome, Tendinitis and Repetitive motion injuries.', 'score': 0.